# Model - XGRBoost

In [26]:
import sys
import os
import yfinance as yf

import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV

root_path = os.path.abspath("..")   # go up one directory
sys.path.append(root_path)

from Project.param import *
from Project.data import prepare_stock_data, load_and_prepare_data

In [2]:
df = yf.download(TICKER, start=START_DATE, end=END_DATE)
df.head()

/var/folders/8y/wwgq01wx345gslv488wqly7m0000gn/T/ipykernel_30571/138827824.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(TICKER, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.418386,6.433082,6.369501,6.400991,493729600
2010-01-05,6.429479,6.465768,6.395589,6.436077,601904800
2010-01-06,6.327211,6.454973,6.320613,6.429480,552160000
2010-01-07,6.315513,6.358101,6.269627,6.350603,477131200
2010-01-08,6.357502,6.358102,6.269928,6.307117,447610800


In [3]:
drop_columns = ['Volume_PPH', 'Open_PPH', 'Low_PPH', 'High_PPH']

X_train, y_train, X_valid, y_valid, X_test, y_test= prepare_stock_data(
    ticker=TICKER,
    start=START_DATE,
    end=END_DATE,
    target_col=TARGET,
    drop_cols=drop_columns
)

/Users/hippolyteheger/code/ArnaudThs/StockProphet/StockProphet/Project/data.py:33: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start, end=end)
[*********************100%***********************]  1 of 1 completed


In [11]:
X_train.columns

Index(['High_AAPL', 'Low_AAPL', 'Open_AAPL', 'Volume_AAPL', 'EMA_9', 'SMA_5',
       'SMA_10', 'SMA_15', 'SMA_30', 'MACD', 'MACD_signal', 'RSI'],
      dtype='object')

2365     50.256866
2366     49.929813
2367     50.550510
2368     49.769871
2369     48.428219
           ...    
2777    113.494820
2778    115.889519
2779    119.462051
2780    119.812515
2781    119.812515
Name: Close_AAPL, Length: 417, dtype: float64

In [23]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train[['EMA_9', 'SMA_5',
       'SMA_10', 'SMA_15', 'SMA_30', 'MACD', 'MACD_signal', 'RSI']])
X_test_scaled = scaler.transform(X_test[['EMA_9', 'SMA_5',
       'SMA_10', 'SMA_15', 'SMA_30', 'MACD', 'MACD_signal', 'RSI']])


In [29]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [31]:
y_train_scaled = scaler.fit_transform(y_train)
y_test_scaled = scaler.transform(y_test)

In [32]:
X_train_scaled

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        5.65713633e-01, 5.67552917e-01, 7.84526454e-01],
       [1.08585660e-03, 7.11532858e-04, 1.08394044e-03, ...,
        5.67049609e-01, 5.69164606e-01, 7.89176035e-01],
       [2.16681377e-03, 1.79133718e-03, 2.17272422e-03, ...,
        5.66218186e-01, 5.70264933e-01, 8.48172454e-01],
       ...,
       [9.95732264e-01, 9.68463999e-01, 9.83238017e-01, ...,
        5.26911756e-01, 6.24067290e-01, 2.96792186e-01],
       [9.94718736e-01, 9.65268402e-01, 9.80710913e-01, ...,
        4.87419607e-01, 5.96276868e-01, 2.08410277e-01],
       [9.91780058e-01, 9.59897450e-01, 9.76305445e-01, ...,
        4.32013371e-01, 5.61451230e-01, 1.65489110e-01]], shape=(1948, 8))

In [34]:
%%time


model = xgb.XGBRegressor(n_estimators=  400,
    learning_rate = 0.05,
    max_depth = 8,
    gamma = 0.005,
    random_state = 42, objective='reg:squarederror')
eval_set = [(X_train_scaled, y_train_scaled), (X_valid, y_valid)]

model.fit(X_train_scaled, y_train_scaled, eval_set=eval_set, verbose=False)
''''
parameters = {
    'n_estimators': [100, 200, 300, 400],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'max_depth': [8, 10, 12, 15],
    'gamma': [0.001, 0.005, 0.01, 0.02],
    'random_state': [42]
}

eval_set = [(X_train, y_train), (X_valid, y_valid)]
model = xgb.XGBRegressor(eval_set=eval_set, objective='reg:squarederror', verbose=False)
clf = GridSearchCV(model, parameters)

clf.fit(X_train, y_train)

print(f'Best params: {clf.best_params_}')
print(f'Best validation score = {clf.best_score_}')
'''

XGBoostError: [17:46:32] /Users/runner/work/xgboost/xgboost/src/data/quantile_dmatrix.cc:58: Check failed: ref->Info().num_col_ == n_features (8 vs. 12) : Invalid ref DMatrix, different number of features.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001480052d8 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000014820c180 xgboost::data::GetCutsFromRef(xgboost::Context const*, std::__1::shared_ptr<xgboost::DMatrix>, unsigned int, xgboost::BatchParam, xgboost::common::HistogramCuts*) + 388
  [bt] (2) 3   libxgboost.dylib                    0x0000000148211c0c xgboost::data::cpu_impl::MakeSketches(xgboost::Context const*, xgboost::data::DataIterProxy<void (void*), int (void*)>*, xgboost::data::DMatrixProxy*, std::__1::shared_ptr<xgboost::DMatrix>, float, xgboost::common::HistogramCuts*, xgboost::BatchParam const&, xgboost::MetaInfo const&, xgboost::data::ExternalDataInfo const&, std::__1::vector<xgboost::Fea

In [6]:
'''
model = xgb.XGBRegressor(**clf.best_params_, objective='reg:squarederror')
model.fit(X_train, y_train, eval_set=eval_set, verbose=False)
'''

"\nmodel = xgb.XGBRegressor(**clf.best_params_, objective='reg:squarederror')\nmodel.fit(X_train, y_train, eval_set=eval_set, verbose=False)\n"

In [7]:
import numpy as np

y_pred = model.predict(X_test)
print(f'y_true = {np.array(y_test)[:5]}')
print(f'y_pred = {y_pred[:5]}')


y_true = [50.25686646 49.92981339 50.55051041 49.76987076 48.42821884]
y_pred = [36.602673 36.23986  36.65404  36.72857  36.653748]


In [8]:
from sklearn.metrics import mean_squared_error

print(f'mean_squared_error = {mean_squared_error(y_test, y_pred)}')


mean_squared_error = 2013.5164212375544


In [10]:
predicted_prices

,Date,High_AAPL,Low_AAPL,Open_AAPL,Volume_AAPL,EMA_9,SMA_5,SMA_10,SMA_15,SMA_30,MACD,MACD_signal,RSI,Predicted,Actual
0,2365,48.557128,47.532989,48.475961,186139600.0,47.962819,49.121962,48.782973,48.392095,47.089200,1.024198,1.208349,50.140482,36.602673,50.256866
1,2366,51.400368,49.948906,50.104081,259309200.0,47.957078,48.796816,48.816870,48.410716,47.201720,1.094002,1.185480,72.275709,36.239861,49.929813
2,2367,50.765359,49.686314,50.094536,127985200.0,48.187056,48.957239,48.993289,48.568276,47.379651,1.110136,1.170411,68.855476,36.654041,50.550510
3,2368,50.571995,50.187644,50.345205,83569600.0,48.361332,49.142014,49.119575,48.730610,47.491535,1.159639,1.168257,71.220446,36.728569,49.769871
4,2369,49.855813,48.581010,48.769603,129772400.0,48.580250,49.497718,49.307930,48.935597,47.656257,1.122935,1.159192,64.091575,36.653748,48.428219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,2777,113.650556,112.112499,112.482416,76499200.0,113.566825,113.120996,114.596754,114.641209,113.102798,0.165095,0.459517,42.382529,37.120518,113.494820
413,2778,114.370928,113.134647,113.475354,46691300.0,113.505109,112.612856,114.287196,114.459822,112.957686,0.149343,0.397482,50.862670,37.031189,115.889519
414,2779,117.758546,113.708980,113.864736,169410200.0,113.504080,112.466838,114.027286,114.323538,112.885488,0.326329,0.383252,58.671235,37.187416,119.462051
415,2780,120.192141,116.823997,117.797449,127728200.0,113.742624,113.479234,113.905605,114.500708,112.991587,0.746263,0.455854,59.017284,37.583344,119.812515


In [9]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Prepare predicted_prices DataFrame
predicted_prices = X_test.copy()
predicted_prices['Predicted'] = y_pred
predicted_prices['Actual'] = y_test.values

# Use the index as Date
predicted_prices = predicted_prices.reset_index()  # moves index to a column called 'index'
predicted_prices.rename(columns={'index': 'Date'}, inplace=True)

# Create figure
fig = make_subplots(rows=1, cols=1)

# True prices
fig.add_trace(go.Scatter(
    x=predicted_prices['Date'],
    y=predicted_prices['Actual'],
    name='Actual',
    mode='lines',
    line=dict(color='LightSkyBlue')
))

# Predicted prices
fig.add_trace(go.Scatter(
    x=predicted_prices['Date'],
    y=predicted_prices['Predicted'],
    name='Predicted',
    mode='lines',
    line=dict(color='MediumPurple')
))

# Layout
fig.update_layout(
    title="Stock Price: Actual vs Predicted",
    xaxis_title="Date",
    yaxis_title="Price",
    legend=dict(x=0, y=1),
    template="plotly_white",
    width=900,
    height=500
)

fig.show()
